In [8]:
# Import necessary libraries
import os
import torch
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import cv2
from sklearn.metrics import classification_report
from pathlib import Path

In [1]:
# Import necessary libraries
import os
import torch
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import cv2
from sklearn.metrics import classification_report
from pathlib import Path

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Specify directories
train_dir = "../data/Training"
test_dir = "../data/Testing"

# Transformations for the dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
])

# Load datasets
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)

# Split train dataset into training and validation
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Load pre-trained ResNet model
resnet_model = models.resnet50(pretrained=True)
criterion = nn.CrossEntropyLoss()


    
num_features = 4
resnet_model.fc = nn.Linear(resnet_model.fc.in_features, num_features)
optimizer = optim.SGD(resnet_model.fc.parameters(), lr=0.001, momentum=0.9)

resnet_model = resnet_model.to(device)

for param in resnet_model.parameters():
    param.requires_grad = False


for param in resnet_model.fc.parameters():
    param.requires_grad = True

# Fine-tuning loop
epochs = 20  # Fine-tune for 20 epochs
for epoch in range(epochs):
    resnet_model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Zero gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = resnet_model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Track loss and accuracy
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {100 * correct / total:.2f}%")

# Save the fine-tuned model
checkpoint_path = "/temp/tumor-detection/checkpoints/resnet/resnet_full.pth"
#checkpoint_dir = os.path.dirname(checkpoint_path)
save_dir = os.path.expanduser('~') + checkpoint_path
#checkpoint_path = "./checkpoint/resnet/resnet_trained.weights.h5"
#checkpoint_dir = os.path.dirname(checkpoint_path)
torch.save(resnet_model, save_dir)
print("Fine-tuned model saved successfully!")

# Testing loop with classification report
resnet_model.eval()
test_correct = 0
test_total = 0
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = resnet_model(images)
        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

        # Store predictions and labels for classification report
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Calculate accuracy
print(f"Test Accuracy: {100 * test_correct / test_total:.2f}%")

/home/zzhen268/anaconda3/envs/ai2env/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/zzhen268/anaconda3/envs/ai2env/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/20], Loss: 0.7451, Accuracy: 74.22%
Epoch [2/20], Loss: 0.4524, Accuracy: 85.23%
Epoch [3/20], Loss: 0.3990, Accuracy: 87.15%
Epoch [4/20], Loss: 0.3606, Accuracy: 87.61%
Epoch [5/20], Loss: 0.3376, Accuracy: 88.66%
Epoch [6/20], Loss: 0.3243, Accuracy: 89.03%
Epoch [7/20], Loss: 0.3157, Accuracy: 89.28%
Epoch [8/20], Loss: 0.3081, Accuracy: 89.03%
Epoch [9/20], Loss: 0.2827, Accuracy: 90.28%
Epoch [10/20], Loss: 0.2815, Accuracy: 90.11%
Epoch [11/20], Loss: 0.2737, Accuracy: 90.50%
Epoch [12/20], Loss: 0.2576, Accuracy: 91.33%
Epoch [13/20], Loss: 0.2603, Accuracy: 90.83%
Epoch [14/20], Loss: 0.2599, Accuracy: 90.79%
Epoch [15/20], Loss: 0.2566, Accuracy: 91.31%
Epoch [16/20], Loss: 0.2508, Accuracy: 91.16%
Epoch [17/20], Loss: 0.2538, Accuracy: 91.11%
Epoch [18/20], Loss: 0.2519, Accuracy: 90.90%
Epoch [19/20], Loss: 0.2501, Accuracy: 91.20%
Epoch [20/20], Loss: 0.2444, Accuracy: 91.14%
Fine-tuned model saved successfully!
Test Accuracy: 87.64%


In [5]:
print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {100 * correct / total:.2f}%")
print(f"Test Accuracy: {100 * test_correct / test_total:.2f}%")

Epoch [20/20], Loss: 0.2444, Accuracy: 91.14%
Test Accuracy: 87.64%


In [6]:
# Generate classification report
report = classification_report(all_labels, all_preds, target_names=train_dataset.dataset.classes)
print("Classification Report:")
print(report)

Classification Report:
              precision    recall  f1-score   support

      glioma       0.90      0.84      0.87       300
  meningioma       0.81      0.71      0.75       306
     notumor       0.90      0.96      0.93       405
   pituitary       0.88      0.98      0.93       300

    accuracy                           0.88      1311
   macro avg       0.87      0.87      0.87      1311
weighted avg       0.87      0.88      0.87      1311



In [12]:
ckpt_parent_path = "/temp/tumor-detection/checkpoints/resnet/"
checkpoint_path = "/temp/tumor-detection/checkpoints/resnet/resnet_full.pth"
#checkpoint_dir = os.path.dirname(checkpoint_path)
parent_dir = os.path.expanduser('~') + ckpt_parent_path
save_dir = os.path.expanduser('~') + checkpoint_path
#checkpoint_path = "./checkpoint/resnet/resnet_trained.weights.h5"
#checkpoint_dir = os.path.dirname(checkpoint_path)
Path(parent_dir).mkdir(parents=True, exist_ok = True)
torch.save(resnet_model, save_dir)

In [8]:
torch.save({
            'epoch': epoch,
            'model_state_dict': efficientnet_model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': running_loss,
            }, os.path.expanduser('~') + checkpoint_path + "_full")

In [11]:
torch.save(optimizer.state_dict(), os.path.expanduser('~') + checkpoint_path + "_opt")